# Ensemble Learning Neural Network for WSN-DS Dataset

This notebook demonstrates how to train an ensemble of neural networks on the WSN-DS dataset using focal loss as the loss function. The workflow includes data loading, preprocessing, model definition, ensemble training, and evaluation.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import confusion_matrix, precision_score


In [ ]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-7
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_sum(loss, axis=1)
    return focal_loss_fixed

In [ ]:
df = pd.read_csv('data/WSN-DS.csv')

y = df.iloc[:, -1]
X = df.iloc[:, :-1]

if y.dtype == 'O':
    y = pd.factorize(y)[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def build_model(input_dim, num_classes):
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss=focal_loss(gamma=2., alpha=.25),
                  metrics=['accuracy'])
    return model

num_classes = len(np.unique(y_train))

In [ ]:
class KerasClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, num_classes, epochs=10, batch_size=32, verbose=0):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model_ = None
    def fit(self, X, y):
        y_cat = keras.utils.to_categorical(y, num_classes=self.num_classes)
        self.model_ = build_model(self.input_dim, self.num_classes)
        self.model_.fit(X, y_cat, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self
    def predict(self, X):
        preds = self.model_.predict(X)
        return np.argmax(preds, axis=1)

ensemble = BalancedBaggingClassifier(
    estimator=KerasClassifierWrapper(input_dim=X_train.shape[1], num_classes=num_classes, epochs=10),
    n_estimators=5,
    random_state=42,
    n_jobs=1
)
ensemble.fit(X_train, y_train)

/Users/biru/Documents/TugasAkhir/.venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/biru/Documents/TugasAkhir/.venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/biru/Documents/TugasAkhir/.venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/Users/biru/Documents/TugasAkhir/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as th

BalancedBaggingClassifier(estimator=KerasClassifierWrapper(input_dim=18,
                                                           num_classes=5),
                          n_estimators=5, n_jobs=1, random_state=42)

In [ ]:
y_pred = ensemble.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 200us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 200us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 197us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 197us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 193us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 193us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step
2342/2342 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step
Accuracy: 0.9737365379739233
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     68014
           1       0.88      1.00      0.94       662
           2       0.92      0.93      0.92      1328
           3       0.73      0.68      0.70      2919
           4       0.68      0.97      0.80      2010

    accuracy                           0.97     74933
   macro avg       0.84      0.91      0.87     74933
weighted avg       0.98      0.97      0.97     74933

Accuracy: 0.9737365379739233
   

In [ ]:
all_X = scaler.transform(X)
all_y = y
all_pred = ensemble.predict(all_X)

print('--- Evaluation on Entire Dataset ---')
print('Accuracy:', accuracy_score(all_y, all_pred))
print(classification_report(all_y, all_pred))
calculate_metrics(all_y, all_pred)

11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 201us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 201us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 189us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 189us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 190us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 190us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 188us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 188us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 185us/step
11709/11709 ━━━━━━━━━━━━━━━━━━━━ 2s 185us/step
--- Evaluation on Entire Dataset ---
Accuracy: 0.9739017405067515
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    340066
           1       0.88      1.00      0.94      3312
           2       0.91      0.93      0.92      6638
           3       0.73      0.68      0.71     14596
           4       0.68      0.98      0.80     10049

    accuracy                           0.97    374661
   macro avg       0.84      0.92      0.87    374661
weighted avg       0.98      0